In [2]:


import numpy as np
import pandas as pd
import quant_utils.data_moudle as dm
from dateutil.parser import parse
from quant_utils.performance import Performance


class PortfolioBacktest:
    def __init__(self, df_holding: pd.DataFrame, end_date: str) -> None:
        """
        初始化

        Parameters
        ----------
        df_holding : pd.DataFrame
            基金不同时间点的持仓信息
            要求columns=["TRADE_DT", "TICKER_SYMBOL", "START_WEIGHT"]
        end_date : str
            净值计算的结束日期
        """
        self.df_holding = df_holding
        self.df_holding["TRADE_DT"] = self.df_holding["TRADE_DT"].apply(
            lambda s : parse(str(s)).strftime("%Y%m%d")
        )
        self.end_date = end_date
        self.start_date = self.df_holding["TRADE_DT"].min()
        self.change_dts = list(self.df_holding["TRADE_DT"].unique())

        # 获取持仓当中需要的代码的净值
        ticker_list = list(set(self.df_holding["TICKER_SYMBOL"].tolist()))
        self.fund_ret = dm.get_fund_return(
            ticker_symbol=ticker_list,
            start_date=self.start_date,
            end_date=self.end_date 
        )
        self.fund_ret["TRADE_DT"] = self.fund_ret["TRADE_DT"].apply(
            lambda s: s.strftime("%Y%m%d")
        )

    def update_dt_holding(self, dt_holding: pd.DataFrame) -> pd.DataFrame:
        """
        更新单日持仓

        Parameters
        ----------
        dt_holding : pd.DataFrame
            单日持仓，要求columns=["TRADE_DT", "TICKER_SYMBOL", "START_WEIGHT"]

        Returns
        -------
        pd.DataFrame
            columns=[
                "TRADE_DT", "TICKER_SYMBOL", "START_WEIGHT", 
                "RETURN_RATE", "LOG_RETURN_RATE", "WEIGHT_RETURN", "END_WEIGHT", 
            ]
        """
        dt_holding = dt_holding.copy()
        dt_holding = dt_holding.merge(
            self.fund_ret, 
            how='left', 
            on=["TRADE_DT", "TICKER_SYMBOL"]
        )
        dt_holding["WEIGHT_RETURN"] = dt_holding["START_WEIGHT"] * dt_holding["RETURN_RATE"]/100
        dt_holding["END_WEIGHT"] = 100 * (
            dt_holding["START_WEIGHT"] * (1+dt_holding["RETURN_RATE"]/100)
        )/np.sum((dt_holding["START_WEIGHT"] * (1+dt_holding["RETURN_RATE"]/100)))
        return dt_holding

    def update_df_holding(self)->pd.DataFrame:
        """
        更新每天的持仓信息，补全
        "RETURN_RATE", "LOG_RETURN_RATE", "WEIGHT_RETURN", "END_WEIGHT"

        Returns
        -------
        pd.DataFrame
            columns=[
                "TRADE_DT", "TICKER_SYMBOL", "START_WEIGHT", 
                "RETURN_RATE", "LOG_RETURN_RATE", "WEIGHT_RETURN", "END_WEIGHT", 
            ]
        """
        trade_dts = dm.get_period_end_date(
            start_date=self.start_date,
            end_date=self.end_date,
            period="d"
        )
       # 更新初始的上一个交易日及持仓
        last_trade_dt = self.change_dts[0]
        last_holding = self.df_holding.query(
            f"TRADE_DT == '{last_trade_dt}'"
        )
        last_holding = self.update_dt_holding(last_holding)
        df_holding_list = [last_holding]
        # 遍历交易日,并补全持仓
        for trade_dt in trade_dts[1:]:
            # 如果不是调仓日,则取上一交易日结束权重
            if trade_dt not in self.change_dts:
                temp_holding = last_holding.copy()
                temp_holding["TRADE_DT"] = trade_dt
                temp_holding["START_WEIGHT"] = temp_holding["END_WEIGHT"]
                temp_holding = temp_holding[["TRADE_DT", "TICKER_SYMBOL", "START_WEIGHT"]]
            # 如果是交易日则按照最新持仓补全
            else:
                temp_holding = self.df_holding.query(
                    f"TRADE_DT == '{trade_dt}'"
                )
            
            temp_holding = self.update_dt_holding(temp_holding)
            df_holding_list.append(temp_holding)
            last_holding = temp_holding.copy()
        self.dts_holding = pd.concat(df_holding_list)
    
    def update_portfolio_performance(self)->None:
        """
        更新组合表现
        """
        # 更新组合每日持仓
        self.update_df_holding()

        # 获取每日收益率
        self.portfolio_rets = (
            self.dts_holding
            .groupby(by="TRADE_DT")["WEIGHT_RETURN"].sum()
            .reset_index()
            .rename(columns={"WEIGHT_RETURN": "RETURN_RATE"})
        )
        self.portfolio_rets["LOG_RETURN_RATE"] = 100 * np.log(1+self.portfolio_rets["RETURN_RATE"]/100)
        
        # 计算每日净值，并归一
        self.portfolio_nav = (1 + self.portfolio_rets["RETURN_RATE"]/100).cumprod()
        self.portfolio_nav = self.portfolio_nav/self.portfolio_nav[0]
        self.portfolio_nav.index = self.portfolio_rets["TRADE_DT"]
        self.portfolio_nav.name = "NAV"
        
        # 计算组合表现绩效
        self.portfolio_performance = Performance(self.portfolio_nav).stats().T
    

In [3]:
if __name__ == "__main__":
    df = FundPerformance("20230822", "20230821")
    dates_df = df.get_needed_dates_df()

20230821
20230822


C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_15684\3224070936.py:168: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return dates_df.append(temp_fund_info_result).drop_duplicates()


In [22]:
# def covert_to_dict(df):
#     for df.iterrows():
dates_dict = {}
for k,v in dates_df.iterrows():
    if v["TICKER_SYMBOL"] not in dates_dict.keys():
        dates_dict[v["TICKER_SYMBOL"]] = []
    dates_dict[v["TICKER_SYMBOL"]].append((v["START_DATE"], v["END_DATE"]))

In [35]:
for _,(start_date, end_date) in dates_df.query("TICKER_SYMBOL == '110011'")[["START_DATE", "END_DATE"]].iterrows():
    print(start_date, end_date)

20230505 20230821
20230221 20230821
20221230 20230821
20221121 20230821
20230504 20230821
20230630 20230821
20230519 20230821
20230317 20230821
20230621 20230821
20220819 20230821
20230731 20230821
20220722 20230821
20220124 20230821
20230721 20230821
20230814 20230821
20230804 20230821
20230821 20230821
20230817 20230821
20230818 20230821
20211015 20230821
20230505 20230822
20221230 20230822
20230504 20230822
20230630 20230822
20230519 20230822
20230317 20230822
20230822 20230822
20230222 20230822
20230621 20230822
20230815 20230822
20230731 20230822
20220722 20230822
20230522 20230822
20220124 20230822
20221122 20230822
20220822 20230822
20230721 20230822
20230804 20230822
20230821 20230822
20230817 20230822
20230818 20230822
20211015 20230822
20210910 20230821
20210910 20230822
